In [9]:
import cv2
from PIL import Image
import fitz

In [10]:
image = Image.open("3.png")
image.show()

In [1]:
image_path = "3.png"

try:
  doc = fitz.open(image_path)
  page = doc[0]
  paths = page.get_drawings()

  outpdf = fitz.open()
  outpage = outpdf.new_page(width=page.rect.width, height=page.rect.height)
  shape = outpage.new_shape()

for path in paths:
    for item in path["items"]:
      if item[0] == "l":
        shape.draw_line(item[1], item[2])
      elif item[0] == "re":
        shape.draw_rect(item[1])
      elif item[0] == "qu":
        shape.draw_quad(item[1])
      elif item[0] == "c":
        shape.draw_bezier(item[1], item[2], item[3], item[4])
      else:
        raise ValueError("unhandled drawing", item)
shape.finish()

except (fitz.fitz.NonPDFException, OSError) as e:
  print(f"Error opening image with fitz: {e}")
  print("Assuming raster image...")

try:
  with Image.open(image_path) as img:
    # Replace with your processing logic for raster images
    # This section depends on the type of drawings in your image

except (IOError, OSError) as e:
  print(f"Error opening image: {e}")
  exit()

In [2]:
import fitz
import cv2
import os
import numpy as np

image_path = "3.png"

try:
  doc = fitz.open(image_path)
  page = doc[0]
  paths = page.get_drawings()

  outpdf = fitz.open()
  outpage = outpdf.new_page(width=page.rect.width, height=page.rect.height)
  shape = outpage.new_shape()

  for path in paths:
    for item in path["items"]:
      if item[0] == "l":
        shape.draw_line(item[1], item[2])
      elif item[0] == "re":
        shape.draw_rect(item[1])
      elif item[0] == "qu":
        shape.draw_quad(item[1])
      elif item[0] == "c":
        shape.draw_bezier(item[1], item[2], item[3], item[4])
      else:
        raise ValueError("unhandled drawing", item)
  shape.finish()

except (fitz.fitz.NonPDFException, OSError) as e:
  print(f"Error opening image with fitz: {e}")
  print("Assuming raster image...")

  try:
    with Image.open(image_path) as img:
      # Replace with your processing logic for raster images
      # This section depends on the type of drawings in your image

      # Convert the image to grayscale
      gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

      # Apply adaptive thresholding to remove numbers
      thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

      # Save the processed image to a new PDF
      temp_img = Image.fromarray(thresh)
      temp_img.save("processed_image.png")
      temp_doc = fitz.open("processed_image.png")
      outpdf.insert_pdf(temp_doc)
      outpdf.save("output_with_no_numbers.pdf")

      # Remove the temporary image and PDF files
      os.remove("processed_image.png")
      temp_doc.close()

  except (IOError, OSError) as e:
    print(f"Error opening image: {e}")
    exit()

# Find the number of edges in the processed image
try:
  temp_doc = fitz.open("output_with_no_numbers.pdf")
  temp_page = temp_doc[0]
  temp_img = np.array(temp_page.get_pixmap())

  # Convert the image to grayscale
  gray = cv2.cvtColor(temp_img, cv2.COLOR_RGB2GRAY)

  # Apply Canny edge detection
  edges = cv2.Canny(gray, 100, 200)

  # Count the number of edges
  edge_count = np.sum(edges == 255)

  print(f"Number of edges: {edge_count}")

except (OSError) as e:
  print(f"Error opening image: {e}")
  exit()

In [36]:
import cv2
import numpy as np

# Load the image
image_path = "3.png"
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('Gray', gray)

# Remove numbers using adaptive thresholding
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Dilate the pixels around the numbers
kernel = np.ones((5,5), np.uint8)
dilated = cv2.dilate(thresh, kernel, iterations=2)

# Invert the image to have lines as white
inverted = cv2.bitwise_not(dilated)

# Find contours of the lines
contours, _ = cv2.findContours(inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on a blank image
result = np.zeros_like(image)
cv2.drawContours(result, contours, -1, 0, 1)

# Display or save the resulting image
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Converting the numbers to patches

In [1]:
import cv2
import numpy as np

# Load the image
image_path = "3.png"
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold the grayscale image
_, thresh = cv2.threshold(gray, 3, 255, cv2.THRESH_BINARY)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Set a threshold for contour area
area_threshold = 3

# Filter contours representing numbers based on area
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > area_threshold]

# Create patches (rectangles) around the contours representing numbers
for contour in filtered_contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), -1)  # Fill rectangle with white

# Display or save the resulting image
cv2.imshow("Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Extracting Lines and removing any curves

In [ ]:
 # Applying gaussian blur to improve clarity
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image = cv2.imread('2.png', 0)

# Apply Gaussian blurring to reduce noise
image = cv2.GaussianBlur(image, (5, 5), 0)

# Applying Bilateral Filtering
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# gray = cv2.bilateralFilter(gray, 20, 15, 18)
# cv2.imshow('Post-Blur', gray)

#canny_image = cv2.Canny(gray, 23, 55)
# cv2.imshow('canny-image', canny_image)

# Apply adaptive thresholding to get a binary image
_, thresh = cv2.threshold(image, 142, 255, cv2.THRESH_BINARY_INV)
#3.png : 75, 2.png : 142-150

# Apply morphological operations to remove noise and fill in gaps
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an all-white image
output = np.ones_like(image) * 255

for contour in contours:
    # Calculate the area of the contour to identify numbers and lines separately based on their size
    area = cv2.contourArea(contour)
    
    if area < 40:  # This threshold value may need adjustment depending on the specific image and text size.
        continue
    
    # Draw detected contours (lines) on the white background
    cv2.drawContours(output, [contour], -1, (0), 1)

#Plotting
plt.figure(figsize=(12, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(output, cmap='gray')
plt.title('Output')

# Plot the image without numbers
# plt.subplot(1, 3, 3)
# plt.imshow(thresh)
# plt.title('Gray', cmap='gray')

# Show the plots
plt.show()


# Display the processed image
cv2.imshow('Processed Image', output)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Removing numbers sticking to bar

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read image
img = cv2.imread('st_bar_main.png')
#cv2.imshow('Original', img)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
canny_image = cv2.Canny(gray, 35, 95)
#cv2.imshow('canny-image', canny_image)

# Find contours of the white regions
contours, hierarchy = cv2.findContours(canny_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Gaussian Blur
gray = cv2.GaussianBlur(gray, (3, 3), 0)

# Getting Better results with bilateral filtering
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 150, 70, 30)
#cv2.imshow('Post-Blur', gray)

# Threshold the image
threshold_value = 150
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 3))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
#cv2.imshow('Smooth', thresh)

# Invert the image
thresh = cv2.bitwise_not(thresh)

# Create a mask image with only the black regions
mask = np.zeros_like(thresh)

for cnt in contours:
    area = cv2.contourArea(cnt)
    if area < 40: # Change this threshold value according to your image
        cv2.drawContours(mask, [cnt], -1, 0, -1)

# Invert the mask
mask = cv2.bitwise_not(mask)

# Apply the mask to the original image
result = cv2.bitwise_and(thresh, thresh, mask=mask)

# Display the extracted lines
#cv2.imshow('Extracted Lines', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

#Plotting
plt.figure(figsize=(16, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(canny_image, cmap='gray')
plt.title('Canny Image')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(result, cmap='gray')
plt.title('Output')

# Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(img, cmap = plt.cm.gray) 
plt.title('Gray')

# Show the plots
plt.show()

# # Save the image
# cv2.imwrite('new_image2.png', result)

## Attempt 1

In [14]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('2.png')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 45, 70, 50)

# Apply Canny edge detection
edges = cv2.Canny(gray, 45, 90)
#cv2.imshow('canny-image', canny_image)

# Find contours of the white regions
contours, hierarchy = cv2.findContours(canny_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Perform Hough Line Transform to detect lines
lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)

# Draw detected lines on a copy of the original image
lines_image = image.copy()
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(lines_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

# Display the original image with detected lines
cv2.imshow('Detected Lines', lines_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Attempt 2

In [10]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('2.png')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 45, 70, 50)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Perform edge detection using Canny
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Perform dilation and erosion to close gaps in between lines
kernel = np.ones((5,5), np.uint8)
closed_edges = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Find contours in the closed edges image
contours, _ = cv2.findContours(closed_edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on a black background
contour_image = np.zeros_like(image)
for contour in contours:
    cv2.drawContours(contour_image, [contour], -1, (255, 255, 255), 2)

# Invert the contour image to create a mask for numbers
mask = cv2.bitwise_not(contour_image)

# Apply the mask to the original image to remove numbers
cleaned_image = cv2.bitwise_and(image, mask)

# Display the cleaned image
cv2.imshow('Cleaned Image', cleaned_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Attempt 3

In [9]:
import cv2
import numpy as np

image_path = "3.png"

# Load the image
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply adaptive thresholding to remove numbers
thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Find contours in the thresholded image
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Initialize an empty image to store the lines
lines_img = np.zeros_like(img)
cv2.imshow('Output', lines_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Iterate through the contours
for contour in contours:
    # Compute the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)

    # Check if the bounding box is too small (i.e., a small noise blob)
    if w * h < 100:
        continue

    # Compute the aspect ratio of the bounding box
    aspect_ratio = float(w) / h

    # Check if the aspect ratio is within a certain range (e.g., between 0.2 and 10)
    if aspect_ratio < 0.2 or aspect_ratio > 10:
        continue

    # Compute the convex hull of the contour
    hull = cv2.convexHull(contour)

    # Check if the convex hull is too small (i.e., a small noise blob)
    if len(hull) < 5:
        continue

    # Compute the moments of the contour
    moments = cv2.moments(contour)

    # Compute the centroid of the contour
    cx = int(moments["m10"] / moments["m00"])
    cy = int(moments["m01"] / moments["m00"])

    # Draw the contour on the lines image
    cv2.drawContours(lines_img, [contour], 0, 255, -1)

    # Draw the centroid on the lines image
    cv2.circle(lines_img, (cx, cy), 3, 255, -1)

# Save the lines image to a file
# cv2.imwrite("lines.png", lines_img)

## Sobel / Laplacian Attempt

In [280]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

image_path = "4.png"

# Load the image
img = cv2.imread(image_path)

# Grayscale Image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 30, 50, 40)
#cv2.imshow('Gray', gray)
#70,30,40 - 50,35,60

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)
           
#Contour Detection
contours, hierarchy = cv2.findContours(lap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#Threshold
threshold_value = 40
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
# 60

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1)) #(2,1)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
#cv2.imshow('Smooth', thresh)

# kernel = np.ones((1, 1), dtype=np.uint8)  # Create a kernel for morphology
# closed_binary = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

dilation_kernel = np.ones((3, 3), np.uint8)
dilation_iterations = 2  # Number of dilation iterations

# Invert the mask to create a mask for numbers
mask = cv2.bitwise_not(thresh)

# Apply the mask to the original image to remove numbers
cleaned_image = cv2.bitwise_and(img, img, mask=mask)

# perimeter approach
perimeter_threshold = 100  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.arcLength(cnt, True) > perimeter_threshold]

white_background = np.ones_like(img) * 255

# Draw contours on the white background
contour_image = cv2.drawContours(white_background, filtered_contours, -1, (0, 0, 0), 2)

for cnt in contours:
    area = cv2.contourArea(cnt)
    if area >= 20:
        cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)

cv2.waitKey(0)
cv2.destroyAllWindows()

#Plotting
plt.figure(figsize=(15, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(mask, cmap='gray')
plt.title('Mask')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(lap, cmap='gray')
plt.title('Edge Detection')

# # Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(closed_binary)
plt.title('Binary')

# Show the plots
plt.show()

plt.show()

### for 4.png

In [115]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

image_path = "st_bar_main.png"

# Load the image
img = cv2.imread(image_path)

# Grayscale Image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 50, 35, 60)
#cv2.imshow('Gray', gray)
#70,30,40 - 50,35,60

#gaussian Blur
#gray = cv2.GaussianBlur(gray, (1, 1), 0)

# Sobel Edge Detection
# sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
# sobely = cv2.Sobel(gray, cv2.CV_64F, 0 ,1)
# combined_sobel = cv2.bitwise_or(sobelx, sobely)
# cv2.imshow('sobel', combined_sobel)

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)
           
#Contour Detection
contours, hierarchy = cv2.findContours(lap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours
# cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
# cv2.imshow('Contours', img)

#Threshold
threshold_value = 60
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
# 60

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1)) #(2,1)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
#cv2.imshow('Smooth', thresh)

kernel = np.ones((1, 1), dtype=np.uint8)  # Create a kernel for morphology
closed_binary = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

# Invert the mask to create a mask for numbers
mask = cv2.bitwise_not(thresh)

# Apply the mask to the original image to remove numbers
cleaned_image = cv2.bitwise_and(img, img, mask=mask)

# perimeter approach
perimeter_threshold = 100  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.arcLength(cnt, True) > perimeter_threshold]

white_background = np.ones_like(img) * 255

# Draw contours on the white background
contour_image = cv2.drawContours(white_background, filtered_contours, -1, (0, 0, 0), 2)

cv2.waitKey(0)
cv2.destroyAllWindows()

#Plotting
plt.figure(figsize=(15, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(mask, cmap='gray')
plt.title('Mask')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(lap, cmap='gray')
plt.title('Edge Detection')

# # Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(closed_binary)
plt.title('Binary')

# Show the plots
plt.show()

plt.show()

In [111]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

image_path = "4.png"

# Load the image
img = cv2.imread(image_path)

# Grayscale Image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 5, 10, 10)
#cv2.imshow('Gray', gray)
#70,30,40 - 50,35,60

#gaussian Blur
#gray = cv2.GaussianBlur(gray, (1, 1), 0)

# Sobel Edge Detection
# sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
# sobely = cv2.Sobel(gray, cv2.CV_64F, 0 ,1)
# combined_sobel = cv2.bitwise_or(sobelx, sobely)
# cv2.imshow('sobel', combined_sobel)

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)
           
#Contour Detection
contours, hierarchy = cv2.findContours(lap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours
# cv2.drawContours(img, contours, -1, 255, 3)
# cv2.imshow('Contours', img)

#Threshold
threshold_value = 60
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
# 60

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1)) #(2,1)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
#cv2.imshow('Smooth', thresh)

# Invert the mask to create a mask for numbers
mask = cv2.bitwise_not(thresh)

# Apply the mask to the original image to remove numbers
cleaned_image = cv2.bitwise_and(img, img, mask=mask)

cv2.waitKey(0)
cv2.destroyAllWindows()

#Plotting
plt.figure(figsize=(15, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(mask, cmap='gray')
plt.title('Mask')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(lap, cmap='gray')
plt.title('Edge Detection')

# # Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(cleaned_image)
plt.title('Binary')

# Show the plots
plt.show()

plt.show()

### Attempt 5

In [79]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image_path = "st_bar_main.png"  # Replace with your image path
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 50, 35, 60)

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)
           
#Contour Detection
contours, hierarchy = cv2.findContours(lap, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# # Apply binary thresholding
# threshold_value = 25 # Adjust as needed
# binary = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY)[1]

threshold_value = 60
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1)) #(2,1)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Invert the image
thresh = cv2.bitwise_not(thresh)

# Morphological closing to fill gaps
kernel_size = (1, 1)  # Adjust as needed
kernel = np.ones(kernel_size, dtype=np.uint8)
closed_binary = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)  # Lower iterations to avoid over-filling

#Dilation
dilated_img = cv2.dilate(thresh, kernel, iterations=4)

# Iterate over all contours
for contour in contours:
    # Compute the perimeter of the contour
    perimeter = cv2.arcLength(contour, closed=True)
    
    # If the perimeter is above the threshold, draw the contour on the new image
    if perimeter > 50:
        cv2.drawContours(thresh, [contour], 0, 255, 2)

# Display the original and cleaned images
plt.figure(figsize=(10, 3))

# Original binary image
plt.subplot(121)
plt.imshow(thresh, cmap="gray")
plt.title("Binary Image")
plt.axis("off")

# Cleaned image with small objects removed
plt.subplot(122)
plt.imshow(dilated_img, cmap="gray")
plt.title("Contours After Cleaning")
plt.axis("off")

plt.tight_layout()
plt.show()


## Getting results for straight bar

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def remove_small_objects (image, min_area):
    """
    Removes small objects from a binary image.
    """
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask for the output image
    mask = np.zeros(image.shape, dtype=np.uint8)

    # Draw only contours that meet or exceed the minimum area
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area >= min_area:
            cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)  # Fill the contour

    return mask

# Load the image
image_path = "st_bar_main.png"  # Replace with your image path
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 12, 40, 20)
#15,60,20
#20,60,20

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)
           
# Apply binary thresholding
threshold_value = 60 # Adjust as needed
binary = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY)[1]

# Morphological closing to fill gaps
kernel_size = (1, 1)  # Adjust as needed
kernel = np.ones(kernel_size, dtype=np.uint8)
closed_binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)  

# Define minimum area to retain (lower for testing)
min_area = 200
0 # Reduced from 200 to 50 to check smaller contours

# Remove small objects
cleaned_image = remove_small_objects(closed_binary, min_area)
cv2.imshow('Output', cleaned_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Display the original and cleaned images
plt.figure(figsize=(12, 6))

# Original binary image
plt.subplot(121)
plt.imshow(closed_binary, cmap="gray")
plt.title("Binary Image")
plt.axis("off")

# Cleaned image with small objects removed
plt.subplot(122)
plt.imshow(closed_binary, cmap="gray")
plt.title("Contours After Cleaning")
plt.axis("off")

plt.tight_layout()
plt.show()


In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def remove_small_objects(image, min_area):
    """
    Removes small objects from a binary image.
    """
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask for the output image
    mask = np.zeros(image.shape, dtype=np.uint8)

    # Draw only contours that meet or exceed the minimum area
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area >= min_area:
            cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)  # Fill the contour

    return mask

# Load the image
image_path = "4.png"  # Replace with your image path
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#gray = cv2.bilateralFilter(gray, 20, 60, 20)
#15,60,20

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)
           
# Apply binary thresholding
threshold_value = 60 # Adjust as needed
binary = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY)[1]

# Morphological closing to fill gaps
kernel_size = (1, 1)  # Adjust as needed
kernel = np.ones(kernel_size, dtype=np.uint8)
closed_binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)  

# Define minimum area to retain (lower for testing)
min_area = 20
0 # Reduced from 200 to 50 to check smaller contours

# Remove small objects
cleaned_image = remove_small_objects(closed_binary, min_area)

# Display the original and cleaned images
plt.figure(figsize=(12, 6))

# Original binary image
plt.subplot(121)
plt.imshow(gray, cmap="gray")
plt.title("Original Image")
plt.axis("off")

# Cleaned image with small objects removed
plt.subplot(122)
plt.imshow(mask, cmap="gray")
plt.title("Contours After Cleaning")
plt.axis("off")

plt.tight_layout()
plt.show()


In [257]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


# Read the image
image = cv2.imread("4.png")


# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


# Apply Canny edge detection to detect edges
edges = cv2.Canny(gray, 25, 100, apertureSize=3)


# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


# Create a copy of the original image to draw contours on
contour_image = image.copy()


# Draw all contours on the image
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)


# Display the original image and the image with highlighted lines
plt.figure(figsize=(12, 6))


plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')


plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
plt.title('Image with Highlighted Lines')
plt.axis('off')


plt.show()

## Using Perimeter Loop

In [250]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread("st_bar_main.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# gray = cv2.bilateralFilter(gray, 15, 60, 20)

# Apply Canny edge detection to detect edges
# edges = cv2.Canny(gray, 50, 150, apertureSize=3)

edges = cv2.Laplacian(gray, cv2.CV_64F)
edges = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)

#Threshold
threshold_value =30
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
# 60

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Threshold for contour perimeter
perimeter_threshold = 300  # Adjust as needed

# Create a copy of the original image to draw contours on
contour_image = image.copy()

# Draw only contours with perimeter above the threshold
for contour in contours:
    perimeter = cv2.arcLength(contour, True)
    if perimeter > perimeter_threshold:
        cv2.drawContours(contour_image, [contour], -1, (0, 255, 0), 2)

#Plotting
plt.figure(figsize=(16, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(thresh, cmap='gray')
plt.title('Canny Image')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(contour_image, cmap='gray')
plt.title('Output')

# Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(image, cmap = plt.cm.gray) 
plt.title('Gray')

# Show the plots
plt.show()

## Using Area Loop

In [214]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread("st_bar_main.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


# Apply Canny edge detection to detect edges
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Threshold for contour area
area_threshold = 100  # Adjust as needed

# Create a mask for contours above the area threshold
mask = np.zeros_like(gray)
for contour in contours:
    area = cv2.contourArea(contour)
    if area > area_threshold:
        cv2.drawContours(mask, [contour], -1, 255, -1)

# Apply the mask to the original image to remove smaller objects
result_image = cv2.bitwise_and(image, image, mask=mask)

# # Display the original image and the result image
# plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.title('Original Image')
# plt.axis('off')

# plt.subplot(1, 2, 2)
# plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
# plt.title('Image after Removing Small Objects (Area Threshold: {})'.format(area_threshold))
# plt.axis('off')

# plt.show()

#Plotting
plt.figure(figsize=(16, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Canny Image')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(result_image, cmap='gray')
plt.title('Output')

# Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(image, cmap = plt.cm.gray) 
plt.title('Gray')

# Show the plots
plt.show()


In [254]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread("4.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 10, 20, 20)

# Apply Canny edge detection to detect edges
# canny = cv2.Canny(gray, 10, 10, apertureSize=3)

edges = cv2.Laplacian(gray, cv2.CV_64F)
edges = np.uint8(np.absolute(lap))

# Apply binary thresholding
threshold_value = 80  # Adjust as needed
binary = cv2.threshold(edges, threshold_value, 255, cv2.THRESH_BINARY)[1]

# Morphological closing to fill gaps
kernel_size = (1, 1)  # Adjust as needed
kernel = np.ones(kernel_size, dtype=np.uint8)
closed_binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Threshold for contour area
area_threshold = 300  # Adjust as needed

# Create a mask for contours above the area threshold
mask = np.zeros_like(gray)
for contour in contours:
    area = cv2.contourArea(contour)
    if area > area_threshold:
        cv2.drawContours(mask, [contour], -1, 255, -1)

# Create a white background image
white_background = np.ones_like(image) * 255

# Apply the mask to the white background image to remove smaller objects
result_image = cv2.bitwise_and(white_background, white_background, mask=mask)

# Invert the mask
mask = cv2.bitwise_not(result_image)

#Plotting
plt.figure(figsize=(16, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Canny Image')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(mask)
plt.title('Output')

# Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(thresh, cmap = plt.cm.gray) 
plt.title('Gray')

# Show the plots
plt.show()

In [259]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread("3.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection to detect edges
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy of the original image to draw contours on
contour_image = image.copy()

# Draw all contours on the image
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

# Display the original image and the image with highlighted lines
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
plt.title('Image with Highlighted Lines')
plt.axis('off')

plt.show()

In [116]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

image_path = "4.png"

# Load the image
img = cv2.imread(image_path)

# Grayscale Image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 25, 70, 60)

#Threshold
threshold_value = 70
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

# Morphological opening to smooth corners
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Remove noise
kernel = np.ones((1, 1), dtype=np.uint8)  # Create a kernel for morphology
closed_binary = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

# Invert the mask to create a mask for numbers
mask = cv2.bitwise_not(thresh)

# Apply the mask to the original image to remove numbers
cleaned_image = cv2.bitwise_and(img, img, mask=mask)

# Edge Detection
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))

# Contour Detection
contours, hierarchy = cv2.findContours(lap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Remove contours with area below a threshold value
threshold_area = 100  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > threshold_area]

white_background = np.ones_like(img) * 255

# Draw contours on the white background
contour_image = cv2.drawContours(white_background, filtered_contours, -1, (0, 0, 0), 2)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Plotting
plt.figure(figsize=(15, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(mask, cmap='gray')
plt.title('Mask')

# Plot the Edge Detection image
plt.subplot(1, 3, 2)
plt.imshow(lap, cmap='gray')
plt.title('Edge Detection')

# Plot the Binary image after morphological opening
plt.subplot(1, 3, 3)
plt.imshow(contour_image)
plt.title('Contours (Area > %d)' % threshold_area)

# Show the plots
plt.show()

## Removing small patches using area threshold

In [11]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

def remove_small_objects(image, min_area):
    """
    Removes small objects from a binary image.
    """
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask for the output image
    mask = np.zeros(image.shape, dtype=np.uint8)

    # Draw only contours that meet or exceed the minimum area
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area >= min_area:
            cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)  # Fill the contour

    return mask

image_path = "st_bar_main.png"

# Load the image
img = cv2.imread(image_path)

# Grayscale Image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 30, 70, 60)
#cv2.imshow('Gray', gray)
#70,30,40 - 50,35,60

#gaussian Blur
#gray = cv2.GaussianBlur(gray, (1, 1), 0)

# Sobel Edge Detection
# sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
# sobely = cv2.Sobel(gray, cv2.CV_64F, 0 ,1)
# combined_sobel = cv2.bitwise_or(sobelx, sobely)
# cv2.imshow('sobel', combined_sobel)

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))
# cv2.imshow('Laplacian', lap)
           
#Contour Detection
contours, hierarchy = cv2.findContours(lap, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours
# cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
# cv2.imshow('Contours', img)

#Threshold
threshold_value = 60
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
# 60

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1)) #(2,1)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
#cv2.imshow('Smooth', thresh)

kernel = np.ones((1, 1), dtype=np.uint8)  # Create a kernel for morphology
closed_binary = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

# Invert the mask to create a mask for numbers
mask = cv2.bitwise_not(thresh)

# Apply the mask to the original image to remove numbers
cleaned_image = cv2.bitwise_and(img, img, mask=mask)

# perimeter approach
perimeter_threshold = 100  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.arcLength(cnt, True) > perimeter_threshold]

white_background = np.ones_like(img) * 255

# Draw contours on the white background
contour_image = cv2.drawContours(white_background, filtered_contours, -1, (0, 0, 0), 2)

cv2.waitKey(0)
cv2.destroyAllWindows()

#Plotting
plt.figure(figsize=(15, 3))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(mask, cmap='gray')
plt.title('Mask')

# Plot the Adaptive Binary image
plt.subplot(1, 3, 2)
plt.imshow(lap, cmap='gray')
plt.title('Edge Detection')

# # Plot the image without numbers
plt.subplot(1, 3, 3)
plt.imshow(closed_binary)
plt.title('Binary')

# Show the plots
plt.show()

plt.show()

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def remove_small_objects(image, min_area):
    """
    Removes small objects from a binary image.
    """
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask for the output image
    mask = np.zeros(image.shape, dtype=np.uint8)

    # Draw only contours that meet or exceed the minimum area
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area >= min_area:
            cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)  # Fill the contour

    return mask

# Load the image
image_path = "4.png"  # Replace with your image path
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 20, 60, 20)
#15,60,20

#Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))

# Apply binary thresholding
threshold_value = 60 # Adjust as needed
binary = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY)[1]

# Morphological closing to fill gaps
kernel_size = (3, 3)  # Adjust as needed
kernel = np.ones(kernel_size, dtype=np.uint8)
closed_binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)  

# Define minimum area to retain (lower for testing)
min_area = 550
0 # Reduced from 200 to 50 to check smaller contours

# Remove small objects
cleaned_image = remove_small_objects(closed_binary, min_area)
#cv2.imshow('Output', cleaned_image)

#cv2.waitkey(0)

# Display the original and cleaned images
plt.figure(figsize=(12, 6))

# Original binary image
plt.subplot(121)
plt.imshow(gray, cmap="gray")
plt.title("Original Image")
plt.axis("off")

# Cleaned image with small objects removed
plt.subplot(122)
plt.imshow(mask, cmap="gray")
plt.title("Contours After Cleaning")
plt.axis("off")

plt.tight_layout()
plt.show()
